In [10]:
import os
import sys
import re
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2016,12)

In [86]:
test_data ="""cpy 41 a
inc a
inc a
dec a
jnz a 2
dec a
"""

In [14]:
test_data

'cpy 41 a\ninc a\ninc a\ndec a\njnz a 2\ndec a\n'

In [88]:
registers = {
    'a': 0,
    'b': 0,
    'c': 0,
    'd': 0
}

instructions = []
instructions.append('inc')
instructions.append('dec')
instructions.append('cpy')
instructions.append('jnz')

pattern_instruction = re.compile(r'(\w+) (\S+) *(\S+)?')
pattern_jump_value = re.compile(r'([+-])?(\d+)')

In [124]:
def decode_program(input_text):
    """
    Function to parse text block the program instructions

    parameters:
        uinput_text: string name of data

    returns:
        program: list of dict, program instructions
    """
    program = []
    # split text into lines
    lines = input_text.split('\n')

    for line in lines:
        matches = pattern_instruction.match(line)
        if matches:
            instruction = matches.group(1)
            if instruction in ['inc','dec']:
                register = matches.group(2)
                program.append({
                    'instruction': instruction,
                    'register': register
                })
            elif instruction in ['cpy']:
                source = matches.group(2)
                if source.isdigit():
                    source = int(source)
                register = matches.group(3)
                program.append({
                    'instruction': instruction,
                    'source': source,
                    'register': register
                })
            elif instruction in ['jnz']:
                val_x = matches.group(2)
                if val_x.isdigit():
                    val_x = int(val_x)
                val_y = matches.group(3)
                matches2 = pattern_jump_value.match(val_y)
                direction = matches2.group(1) or '+'
                val_y = int(matches2.group(2))
                if direction == '-':
                    val_y*=-1
                program.append({
                    'instruction': instruction,
                    'x': val_x,
                    'y': val_y
                })

    return program

In [140]:
def run_program(program):
    """
    Executes the program instructions in PROGRAM
    """
    pointer = 0
    sentinel = 0
    threshold = 100000
    while 0 <= pointer < len(program):
        sentinel+=1
        # don't forget to take this out before a full run, 
        # I submitted the wrong answer and spent 30 minutes tracing output
        # to debug, then noticed that the end of my output file had "runaway?"
        if sentinel > threshold:
            print("runaway?")
            break
        current_line = program[pointer]
        if current_line['instruction'] == 'inc':
            registers[current_line['register']]+=1
            pointer+=1
        elif current_line['instruction'] == 'dec':
            registers[current_line['register']]-=1
            pointer+=1
        elif current_line['instruction'] == 'cpy':
            if isinstance(current_line['source'],int):
                registers[current_line['register']] = current_line['source']
            elif current_line['source'] in registers:
                registers[current_line['register']] = registers[current_line['source']]
            pointer+=1
        elif current_line['instruction'] == 'jnz':
            x_value = current_line['x']
            if not isinstance(current_line['x'],int):
                x_value = registers[current_line['x']]
            if x_value != 0:
                pointer+=current_line['y']
            else:
                pointer+=1
        
        print(f"After: ({current_line}): pointer: {pointer}, registers: {registers}")

In [142]:
my_program = decode_program(test_data)
print(registers)
run_program(my_program)
print(registers)

{'a': 42, 'b': 0, 'c': 0, 'd': 0}
After: ({'instruction': 'cpy', 'source': 41, 'register': 'a'}): pointer: 1, registers: {'a': 41, 'b': 0, 'c': 0, 'd': 0}
After: ({'instruction': 'inc', 'register': 'a'}): pointer: 2, registers: {'a': 42, 'b': 0, 'c': 0, 'd': 0}
After: ({'instruction': 'inc', 'register': 'a'}): pointer: 3, registers: {'a': 43, 'b': 0, 'c': 0, 'd': 0}
After: ({'instruction': 'dec', 'register': 'a'}): pointer: 4, registers: {'a': 42, 'b': 0, 'c': 0, 'd': 0}
After: ({'instruction': 'jnz', 'x': 'a', 'y': 2}): pointer: 6, registers: {'a': 42, 'b': 0, 'c': 0, 'd': 0}
{'a': 42, 'b': 0, 'c': 0, 'd': 0}
